In [56]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import textdistance
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

In [2]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [22]:
def align_words(sent):
    tokens_1 = sent.lower().split()
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    return tokens_1

In [32]:
def tokenizingwords(text):
    tok_text = []
    for sent in text:
        tok_sent = align_words(sent)
        for word in tok_sent:
            tok_text.append(word)
    return tok_text

In [33]:
bad_words = tokenizingwords(bad)
true_words = tokenizingwords(true)

In [18]:
#Vectorizing all words
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)
    
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [38]:
def get_closest_match_vec(text, X, vec, TOPN=5):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [44]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    return similarities.most_common(1)[0]

In [48]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    closestvecs = get_closest_match_vec(text, X, vec)
    closestwordhybr = get_closest_match_with_metric(text, closestvecs, metric)
    return closestwordhybr

In [53]:
pred_words = []
for word in bad_words:
    pred_words.append(get_closest_hybrid_match(word, X, vec)[0])

['пластичнейшими', 'шпионские', 'устройство', 'такой', 'себе', 'лагерный', 'фотоаппарат', 'девушки', 'банд', 'миниатюрная', 'модель', 'камеры', 'sueddeutsche', 'place', 'america', 'апофеозом', 'дня', 'для', 'меня', 'сегодня', 'стала', 'фраза', 'услышанная', 'в', 'новостях', 'поясним', 'эту', 'мысль', 'ополчатся', 'вот', 'такие', 'язычка', 'в', 'массе', 'своей', 'они', 'конечно', 'все', 'чечни', 'милы', 'защищено', 'чавес', 'разве', 'что', 'не', 'соглашусь', 'многие', 'сетуют', 'на', 'отсутствие', 'живого', 'взаимодействия', 'между', 'учеником', 'и', 'учителем', 'а', 'в', 'чем', 'оно', 'по', 'сути', 'основная', 'цель', 'мероприятия', 'практическая', 'отработка', 'навыков', 'по', 'оказанию', 'помощи', 'гражданам', 'пропавшим', 'в', 'дтп', 'а', 'также', 'повышение', 'и', 'совершенствование', 'уровня', 'профессиональной', 'подготовки', 'сотрудников', 'мчс', 'при', 'проведении', 'аварийно-восстановительных', 'работ', 'по', 'ликвидации', 'последствий', 'дорожно-транспортных', 'происшествий',

In [57]:
print("Accuracy: ", accuracy_score(true_words, pred_words))
print('Точность: ', precision_score(true_words, pred_words, average ="macro"))
print("Полнота: ", recall_score(true_words, pred_words, average="macro"))
print("F-мера: ", f1_score(true_words, pred_words, average="macro"))

Accuracy:  0.8294047143427886
Точность:  0.5756987627200234
Полнота:  0.5750321856395535
F-мера:  0.5729562301922348


C:\Users\1255971\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\1255971\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\1255971\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\1255971\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [58]:
len(true_words)

10012

In [59]:
for i in range(0,10012):
    if true_words[i] != pred_words[i]:
        print (true_words[i], pred_words[i])

симпатичнейшее пластичнейшими
шпионское шпионские
гламурный лагерный
бонда банд
superheadz sueddeutsche
clap place
camera america
получатся ополчатся
язычки язычка
очень чечни
милые милы
насчет защищено
чавеса чавес
попавшим пропавшим
аварийно-спасательных аварийно-восстановительных
в общем
общем как
как вы
вы знаете
знаете из
из моего
моего не
недавнего давнего
пропажу продажу
почте.ру т.п
хорошо хорошее
рите тире
потому трампу
что штат
переждать опережать
дубраве дубрава
люминала людмила
повтыкав аффектов
билетным блатным
кассам классам
что-то что
мощный машинный
нерабочем рабочем
кредиток кредит
хорошо хорошее
капулетти капулети
прислужницу пружину
кормилицу кормилица
молодежь молодежи
сегодня средняя
притащиться приобщиться
программист программисты
навешать вешать
оплеух оплеуху
пивоварах правах
конечно канаш
4:0 1:0
начальник начального
по-взрослому взрослому
покрытый покрыты
слег лег
ветрянка наверняка
подсаживается поддерживаться
женщина шпионаж
загруживаюсь загораживаясь
какие-

долго-долго-долго долго-долго
юстировал оставлю
элитному элтону
пульмонологу пульмонологов
насчет нищета
обструктивной обструктивный
эмфиземы эвфемизм
естественно сено
экскурсоводы экскурсовода
значит зачет
субботу будто
кино коне
кидал деклан
кидалы колады
подскажите подскажет
темному местному
зазырить заразить
кровищи кровища
выхи вывихи
люда люд
целью целую
промозгло промозглом
муторно остроумно
в вообще
общем-то то
пекарской краской
натертый раритетный
имбирь сибирь
цедру процедуру
кстати остатке
финдиректор федерико
случайненько случайность
напьешься подашься
дирижаблями дирижабли
насущного смущенного
теоретизированное третирование
чтобы что
больно бы
не больно
было небывало
ешь орешь
созерцать царь
хорошо хорошее
замашками замками
земляничное значениям
обязательно обязательное
сделать возделать
настоящий стоявший
это ето
одна одно
вещей вшей
общая обширная
очень окончательно
горшке горшок
кулака кулак
адски цика
болящих обособляющих
перуанском прекрасном
ило лелио
рамон роман
раз

штабель шталь
сумбур сумбурными
это эта
когда-то когда
заливаюсь валюсь
глянули глянул
стасу статусу
совершенно свершенно
отправленных направленных
мыле мыл
нарезает назревает
непредвиденные непредвиденных
говорю герою
ничего ночи
мишей мешай
когда погода
как-то каток
хотеться отец
убиться биться
веником темников
плинтуса наступали
нравится ницца
камрад кадрам
как какао
как кана
поверхностное поверхностном
неискреннее неискренние
вот водооборотов
позитивно-пьяной позитивной
саб бас
пиццерию пиццерии
погуляла гуляла
понаобещал пообещал
кстати остатке
запариваться развиваться
неуравновешенно уравновешенное
очень очерчен
спрятан спрятана
сильно вольно
влево воевал
покажи пожитки
пожинаем поражением
забегаю забега
болтаю болтают
девчонками девочками
радужном радужно
финбана финна
толкаться откликаться
торбой тобой
придется предаться
парит партии
1930-ые 1930-е
гробить угробить
здесь садитесь
собственно снбо
самые сеаты
как-то катке
изюминку узнику
опять пять
мне мной
свои васи
описывать за